# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.111560e+02     1.463669e+00
 * time: 0.0861058235168457
     1     1.102911e+01     9.070787e-01
 * time: 0.9024248123168945
     2    -1.176678e+01     9.837010e-01
 * time: 1.0432980060577393
     3    -3.390469e+01     7.813146e-01
 * time: 1.1939067840576172
     4    -4.765471e+01     5.598270e-01
 * time: 1.3371779918670654
     5    -5.706556e+01     2.131928e-01
 * time: 1.4814019203186035
     6    -5.989692e+01     1.466281e-01
 * time: 1.5878369808197021
     7    -6.097702e+01     5.500925e-02
 * time: 1.7000329494476318
     8    -6.141350e+01     4.330363e-02
 * time: 1.8117659091949463
     9    -6.165567e+01     4.321450e-02
 * time: 1.9171547889709473
    10    -6.183407e+01     3.049451e-02
 * time: 2.0288658142089844
    11    -6.196883e+01     2.110347e-02
 * time: 2.140554904937744
    12    -6.205206e+01     1.755626e-02
 * time: 2.2464499473571777
    13    -6.209490e+01     1.850567e-02
 * time: 2.35807180404

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671076
    AtomicLocal         -18.8557687
    AtomicNonlocal      14.8522640
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485383 
    Xc                  -19.3336825

    total               -62.261666460348
